# Sequence classification

In this exercise, you will get familiar with how to build RNNs in Keras. You will build a recurrent model to classify moview reviews as either positive or negative.

In [1]:
%matplotlib inline

import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb


Using TensorFlow backend.


## IMDB Sentiment Dataset

The large movie review dataset is a collection of 25k positive and 25k negative movie reviews from [IMDB](http://www.imdb.com). Here are some excerpts from the dataset, both easy and hard, to get a sense of why this dataset is challenging:

> Ah, I loved this movie.

> Quite honestly, The Omega Code is the worst movie I have seen in a very long time.

> The wit and pace and three show stopping Busby Berkley numbers put this ahead of the over-rated 42nd Street. 

> There simply was no suspense, precious little excitement and too many dull spots, most of them trying to show why "Nellie" (Monroe) was so messed up.

The dataset can be found at http://ai.stanford.edu/~amaas/data/sentiment/. Since this is a common dataset for RNNs, Keras has a preprocessed version built-in.

In [2]:
# We will limit to the most frequent 20k words defined by max_features, our vocabulary size
max_features = 20000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)


In [3]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [4]:
y_train[0]

array([0., 1.], dtype=float32)

The data is preprocessed by replacing words with indexes - review [Keras's docs](http://keras.io/datasets/#imdb-movie-reviews-sentiment-classification). Here's the first review in the training set.

In [5]:
review = X_train[5]
review

[1,
 778,
 128,
 74,
 12,
 630,
 163,
 15,
 4,
 1766,
 7982,
 1051,
 2,
 32,
 85,
 156,
 45,
 40,
 148,
 139,
 121,
 664,
 665,
 10,
 10,
 1361,
 173,
 4,
 749,
 2,
 16,
 3804,
 8,
 4,
 226,
 65,
 12,
 43,
 127,
 24,
 15344,
 10,
 10]

We can convince ourselves that these are movies reviews, using the vocabulary provided by keras:

In [6]:
word_index = imdb.get_word_index()

First we create a dictionary from index to word, notice that words are indexed starting from the number 3, while the first three entries are for special characters:

In [7]:
index_word = {i+3: w for w, i in word_index.items()}
index_word[0]=''
index_word[1]='start_char'
index_word[2]='oov'

In [8]:
index_word[4]

'the'

Then we can covert the first review to text:

In [9]:
' '.join([index_word[i] for i in review])

"start_char begins better than it ends funny that the russian submarine crew oov all other actors it's like those scenes where documentary shots br br spoiler part the message oov was contrary to the whole story it just does not mesh br br"

#### Exercise 1 - prepare the data

The reviews are different lengths but we need to fit them into a matrix to feed to Keras. We will do this by picking a maximum word length and cutting off words from the examples that are over that limit and padding the examples with 0 if they are under the limit.

Refer to the [Keras docs](http://keras.io/preprocessing/sequence/#pad_sequences) for the `pad_sequences` function. Use `pad_sequences` to prepare both `X_train` and `X_test` to be `maxlen` long at the most.

In [10]:
from keras.preprocessing.sequence import pad_sequences

In [11]:
maxlen = 80
# Pad and clip the example sequences
X_train_pad = pad_sequences(X_train,maxlen=maxlen)
X_test_pad = pad_sequences(X_test, maxlen=maxlen)

elements in this array are indices-- shorthand for a one-hot encoded tensor of (25000,80,20000), produces (25000,80)

In [12]:
X_train_pad.shape

(25000, 80)

#### Exercise 2 - build an RNN for classifying reviews as positive or negative

Build a single-layer RNN model and train it. You will need to include these parts:

* An `Embedding` layer for efficiently one-hot encoding the inputs - [docs](http://keras.io/layers/embeddings/)
* A recurrent layer. Keras has a [few variants](http://keras.io/layers/recurrent/) you could use. LSTM layers are by far the most popular for RNNs.
* A `Dense` layer for the hidden to output connection.
* A softmax to produce the final prediction.

You will need to decide how large your hidden state will be. You may also consider using some dropout on your recurrent or embedding layers - refers to docs for how to do this.

Training for longer will be much better overall, but since RNNs are expensive to train, you can use 1 epoch to test. You should be able to get > 70% accuracy with 1 epoch. How high can you get?

In [13]:
from keras.layers import LSTM, CuDNNLSTM
from keras.layers import Input, Dense, CuDNNLSTM, Embedding
from keras.models import Model

In [14]:
# Design an recurrent model
# have two outputs, just with tow complimentary classes


#written in functional API:
inputs = Input(shape=(80,))
x = Embedding(input_dim=max_features,
              output_dim=20,
              input_length=maxlen)(inputs)
x = CuDNNLSTM(32)(x)
outputs = Dense(2, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile('rmsprop','categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 80)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 80, 20)            400000    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 32)                6912      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 406,978
Trainable params: 406,978
Non-trainable params: 0
_________________________________________________________________


In [15]:
X_train_pad.shape

(25000, 80)

In [18]:
X_test_pad.shape

(25000, 80)

In [19]:
model.fit(X_train_pad, y_train, batch_size=32, epochs=1, validation_data=(X_test_pad, y_test))
loss, acc = model.evaluate(X_test_pad, y_test, batch_size=32)
print('Test loss:', loss)
print('Test accuracy:', acc)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 4s 171us/step
Test loss: 0.3724726819992065
Test accuracy: 0.84764


In [ ]:

#model written in sequential API
model = Sequential([
    Embedding(input_dim=max_features,
              output_dim=20,
              input_length=80),
    CuDNNLSTM(32)(x),
    Dense(2, activation='softmax')
    
])


model = Model(inputs=inputs, outputs=outputs)
model.summary()


In [ ]:
#what's the baseline?
y_test[:,1]

In [ ]:
import pandas as pd

In [ ]:
pd.Series(y_test[:,1]).value_counts()